# Quanlse Scheduler

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
This tutorial introduces how to use Quanlse's Scheduler to generate pulse sequences for quantum circuits. The outline of this tutorial is as follows:
- Introduction
- Example: generate a pulse sequence to implement a $W$-state using Quanlse‘s Scheduler
- Summary

## Introduction

**Quanlse's Scheduler** is a module that allows for automatic generation of fidelity-optimized and scheduled pulse sequence for a given quantum circuit set to perform a certain quantum computing task.

**Quanlse's Scheduler** has the following benefits:

- Highly automatic: it generates high-fidelity pulses automatically and simultaneously minimizes the overall gate operation time.
- Practical: it considers several limitations of the superconducting quantum system, including leakage errors, decoherence, etc.
- Flexible: it gives users the freedom to customize qubits and circuit parameters. This can also be easily extended to other quantum computing platforms.

**Quanlse's Scheduler** achieves the following goals:

- It generates parameters and AWG input signal arrays for fidelity-optimized pulses when leakage into the state $|2\rangle$ is taken into account.
- It is also capable of scheduling pulses to minimize idle time and therefore reduce decoherence losses.
- At the same time, it allows for the visualization of pulse sequences for the users to check the results.

CNOT gate is rarely directly implemented on superconducting quantum chips. Instead, it is often constructed by piecing together single-qubit gates and other two-qubit gates like CR gate or ISWAP gate that can be easily implemented on a superconducting chip (often called native gates). The two-qubit gates that are available in the transmon-like superconducting qubit architecture can be divided into two categories:

**Flux-controlled**

This class of gates offers the advantage of short gate time to minimize decoherence error. However, tuning the qubit frequency can introduce flux noises and lead to the problem of frequency crowding.

**All-microwave control**

CR gate allows for an all-microwave control, which alleviates the problem of flux noise. However, the much longer time-scale limits the gate fidelity (because of the decoherence effects of qubits).

![cnot](figures/cnot-gate.png)

Since CZ gates can be used to construct a CNOT gate easily by using only two other single-qubit gates, Quanlse's Scheduler offers this way to construct a CNOT gate in a quantum circuit.

## Example: Generate a pulse sequence to implement a $W$-state using Quanlse's Scheduler

In 2000, Dür et al. found that a tripartite system can be entangled in a different way from GHZ (Greenberger-Horne-Zeilinger) state, which they called the $W$-state \[4\]:

$$
|\Phi_{\rm GHZ}\rangle = \frac{1}{\sqrt{2}} (|000\rangle + |111\rangle),
$$

$$
|\Phi_{\rm W}\rangle = \frac{1}{\sqrt{3}} (|001\rangle + |010\rangle + |100\rangle).
$$

Unlike the GHZ state, the remaining $(N-1)$-qubit state is still entangled if one of the qubit is traced out for a prepared $N$-qubit $W$-state. A three-partite $W$-state is a superposition state of three eigenstates, and only one of the particles is excited in each eigenstate. Due to its high robustness against loss, $W$-state received considerable attention in the field of quantum communication.

In this example, we will demonstrate how to generate a $W$-state with Quanlse's Scheduler. Before starting the main steps, we import the numpy dependency:

In [ ]:
from numpy import array

Then we import the dependent packages from Quanlse:

In [ ]:
from Quanlse.QOperation.FixedGate import X, H, CNOT
from Quanlse.QOperation.RotationGate import RY
from Quanlse.Utils.Functions import basis
from Quanlse.Superconduct.Simulator.PulseSim3Q import pulseSim3Q
from Quanlse.Superconduct.SchedulerSupport.PipelineCenterAligned import centerAligned

Before we go on and instantiate a `simulator` object, we usually need to first define the system configurations. For this example, we can use the built-in Quanlse three-qubit simulator `pulseSim3Q`. For more details, please refer to [multi-qubit noisy simulator](https://quanlse.baidu.com/#/doc/tutorial-multi-qubit-noisy-simulator). The system configuration(i.e. qubit frequency, coupling strength, etc) are fixed in this simulator, but the users are able to create a customed simulator object if intended to. We will use a system of the following qubits topology (each with an anharmonicity of $0.33*(2*\pi)\ \textrm{GHz}$).

![cnot](figures/qubitMap.png)

To begin, we need to define the sampling time for the AWG. Then, we can instantiate a `pulseSim3Q` object by passing in the sampling time `dt` we had just defined.

In [ ]:
# Create pulseSim3Q object
dt = 0.01  # AWG sampling time
model = pulseSim3Q(frameMode='lab', dt=dt)
model.savePulse = False
model.pipeline.addPipelineJob(centerAligned)

With the `pulseSim3Q` object defined, we can proceed to constructing the quantum logic circuit. We use the following circuit to generate a $W$-state:

![w](figures/w-circuit.png) 

where,

$$
\begin{align*}
R_1 &= \frac{1}{\sqrt{3}} \begin{bmatrix} \sqrt{2} & -1 \\ 1 & \sqrt{2} \end{bmatrix}, \\
R_2 &= \begin{bmatrix} \cos(\pi/8) & -\sin(\pi/8) \\ \sin(\pi/8) & \cos(\pi/8) \end{bmatrix}, \\
R_3 &= \begin{bmatrix} \cos(\pi/8) & \sin(\pi/8) \\ -\sin(\pi/8) & \cos(\pi/8) \end{bmatrix}.
\end{align*}
$$

As we mentioned before, in superconducting quantum computing, controlled-Z gate is a native gate which can be used to implement a CNOT gate with 2 additional Hadamard gates on the target qubit.

![VQE](figures/cnot-gate-hadamard.png) 

In this example, we use this decomposition method for the CNOT gates. In **Quanlse**, the logic quantum circuit can be defined using the following codes:

In [ ]:
# Define circuit

# R1
RY(-1.231)(model.Q[0])

# X gate
X(model.Q[0])

# CNOT: 0 -> 1
CNOT(model.Q[0], model.Q[1])

# X gate
X(model.Q[0])

# R2
RY(-0.785)(model.Q[2])

# CNOT: 1 -> 2
CNOT(model.Q[1], model.Q[2])

# R3
RY(0.785)(model.Q[2])

# CNOT: 2 -> 1
CNOT(model.Q[2], model.Q[1])

Note that we decompose the rotation gates $R_1, R_2$ and $R_3$ using `U(theta, phi, lambda)`, where
$$
U(\theta, \phi, \lambda) = e^{i(\phi/2+\lambda/2)} R_z(\phi) R_y(\theta) R_z(\lambda) =
\begin{bmatrix} 
    \cos(\theta/2) & - e^{i\lambda} \sin(\theta/2) \\
    e^{i\phi} \sin(\theta/2) & e^{i(\phi + \lambda)} \cos(\theta/2)
\end{bmatrix}.
$$

Then, by calling the `schedule()` method, we obtain a `QJob` object which consists all the information regarding the pulse configurations. Similar to the previous tutorials, we can call the `plot()` function to visualize the complete pulse sequence. Lastly, we can obtain the unitary evolution of the simulation by calling method `runHamiltonian()` which takes a `QHam` object and a `QJob` object.

In [ ]:
# Schedule
job = model.schedule()
job.plot(color=['red', 'green'])
finalState = model.simulate(job=job)[0]["state"].T[0]

The above pulse sequences are produced by Quanlse. When the users run this program on their local devices, Quanlse Cloud Service will return the fidelity of each individual gate that makes up the circuit.

Finally, we can retrieve the final state and visualize the state populations. Knowing that the W state takes form: $|\Phi_{\rm W}\rangle = \frac{1}{\sqrt{3}} (|001\rangle + |010\rangle + |100\rangle)$, we can see below that the pulse sequence above produces a state somewhat close to the ideal W state. The small peaks are caused by noise such as leakage or crosstalk across the qubits.

In [ ]:
# Calculate final state
popList = [abs(item ** 2) for item in finalState]

# Import the operator for generating basis string list
from Quanlse.Utils.Functions import computationalBasisList

# Import the function for plot bar figures
from Quanlse.Utils.Plot import plotBarGraph

# Plot the population of computational basis
basis = computationalBasisList(3, 3)
plotBarGraph(basis, popList, "Population of a W state generated by QuanlseScheduler",
             "Computational Basis", "Population")

## Summary
After reading this tutorial on Quanlse's Scheduler, the users could follow this link [tutorial-scheduler.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-scheduler.ipynb) to the GitHub page of this Jupyter Notebook document and run this program for themselves. The users are encouraged to try implementing other quantum circuits using a similar procedure as this tutorial.

## References

\[1\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://aip.scitation.org/doi/abs/10.1063/1.5089550)

\[2\] https://en.wikipedia.org/wiki/Quantum_optimization_algorithms

\[3\] https://en.wikipedia.org/wiki/Quantum_algorithm

\[4\] [Dür, Wolfgang, Guifre Vidal, and J. Ignacio Cirac. "Three qubits can be entangled in two inequivalent ways." *Physical Review A* 62.6 (2000): 062314.](https://doi.org/10.1103/PhysRevA.62.062314)

\[5\] [Guo, Guang-Can, and Yong-Sheng Zhang. "Scheme for preparation of the W state via cavity quantum electrodynamics." *Physical Review A* 65.5 (2002): 054302.](https://doi.org/10.1103/PhysRevA.65.054302)